# Automation in Data Cleansing 
## Tutorial by *Nikos Gavriil*
-----------------------------------------------------------------------------------------------------------------------------


In this tutorial we are going to talk about automation in the Data Cleansing process. Data cleansing (or data cleaning) is defined in Wikipedia as:

>The process of detecting and correcting (or removing) corrupt or inaccurate records from a record set, table, or database and refers to identifying incomplete, incorrect, inaccurate or irrelevant parts of the data and then replacing, modifying, or deleting the dirty or coarse data.  

So the question is...can it be automated? Yes! Should I do it? To answer this question let me tell you some things about the algorithm to be presented in this project. In order to clean up a dataset you can't just press pandas.DataFrame.dropna()and make the missing values disappear. Actually you can, but the result may not be what you were expecting in the sense that if some columns are full of missing values the whole dataset will be erased. Actually it takes only one missing value per row to delete the whole dataset, so there are some actions to be taken before using the dropna() function. These actions could be:

* _Identifying the columns that are full (or in their largest part full) of NAs and removing them first._ After that if we use the dropna() function less or equal rows will be deleted.
* _Identying useless columns with respect to some task._ For example if I would like to predict one column using the others I could use some criteria to choose the most relevant columns.

As you see, cleaning a dataset includes making some assumptions for the relationship between the features it contains. So other process should be followed if someone expects a linear or nonlinear relationship. The algorithm that will be presented here deals with linear relationship between a variable that will be chosen by the user as dependent and the features that will be used for predicting the dependent variable. So let's move to the mechanics of the algorithm.

In [1]:
#the following python modules are needed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RandomizedLasso
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

Let's break down the main parts of the algorithm...

### Part 1: Preparation

In the first part we prepare the data. Drop the columns that are over 90% covered in missing values, transform the categorical features from strings to integers and standardize the columns of the dataset. 

In [ ]:
le = LabelEncoder()
frame.drop(list(frame.isnull().sum()[frame.isnull().sum()>=int(0.9*frame.shape[0])].index.values),axis=1,inplace=True)
fr = frame.dropna()
for col in fr.columns:
    if ((fr[col].dtype != np.float64) & (fr[col].dtype != np.int64)):
        fr[col] = le.fit_transform(fr[col])
scaler = StandardScaler()
if type(predcolumn)==int: 
    X=fr.drop(fr.columns[predcolumn],axis=1)
elif type(predcolumn)==str:
    X=fr.drop(predcolumn,axis=1)
X = scaler.fit_transform(X)
if type(predcolumn)==int:
    y= fr[fr.columns[predcolumn]]
elif type(predcolumn)==str:
    y= fr[predcolumn]

### Part 2: Relevant features for regression
If the action asked is regression then the algorithm will perform stability selection using Randomised Lasso and keep only the relevant features. Of course we can choose the sensitivity of the algorithm with the alphareg parameters.

In [ ]:
if regression==True:
    model = RandomizedLasso(alpha=alphareg,random_state=1)
    model.fit(X,y)
    filterdcols = model.get_support()
    filterdcols = pd.Series(filterdcols).append(pd.Series(False),ignore_index=True)
    if type(predcolumn)==int:
        framenew = pd.concat([frame[frame.columns[filterdcols]],frame[frame.columns[predcolumn]]],axis=1)
    elif type(predcolumn)==str:
        framenew = pd.concat([frame[frame.columns[filterdcols]],frame[predcolumn]],axis=1)
    framenew = framenew.dropna()

### Part 3: Relevant features for classification
The same philosophy can be applied if the task to be performed is classification. This time the sensitivity is calibrated by the Cclass parameter

In [ ]:
model = RandomizedLogisticRegression(C=Cclass,random_state=1)
model.fit(X,y)
filterdcols = model.get_support()
filterdcols = pd.Series(filterdcols).append(pd.Series(False),ignore_index=True)
if type(predcolumn)==int:
    framenew = pd.concat([frame[frame.columns[filterdcols]],frame[frame.columns[predcolumn]]],axis=1)
elif type(predcolumn)==str:
    framenew = pd.concat([frame[frame.columns[filterdcols]],frame[predcolumn]],axis=1)
framenew = framenew.dropna()

So all you have to do is choose the variable that you would like to predict, the task to be performed (regression or classification) and the sensitivity of the algorithm (controlling how many columns will be left). Connecting all the previous parts:

In [4]:
def cleaner(frame,predcolumn,alphareg ='aic',regression=True,Cclass=1):
    import warnings
    warnings.filterwarnings('ignore')
    le = LabelEncoder()
    frame.drop(list(frame.isnull().sum()[frame.isnull().sum()>=int(0.9*frame.shape[0])].index.values),axis=1,inplace=True)
    fr = frame.dropna()
    for col in fr.columns:
        if ((fr[col].dtype != np.float64) & (fr[col].dtype != np.int64)):
            fr[col] = le.fit_transform(fr[col])
    scaler = StandardScaler()
    if type(predcolumn)==int:
        X=fr.drop(fr.columns[predcolumn],axis=1)
    elif type(predcolumn)==str:
        X=fr.drop(predcolumn,axis=1)
    X = scaler.fit_transform(X)
    if type(predcolumn)==int:
        y= fr[fr.columns[predcolumn]]
    elif type(predcolumn)==str:
        y= fr[predcolumn]
    if regression==True:
        model = RandomizedLasso(alpha=alphareg,random_state=1)
        model.fit(X,y)
        filterdcols = model.get_support()
        filterdcols = pd.Series(filterdcols).append(pd.Series(False),ignore_index=True)
        if type(predcolumn)==int:
            framenew = pd.concat([frame[frame.columns[filterdcols]],frame[frame.columns[predcolumn]]],axis=1)
        elif type(predcolumn)==str:
            framenew = pd.concat([frame[frame.columns[filterdcols]],frame[predcolumn]],axis=1)
        framenew = framenew.dropna()
    else:
        model = RandomizedLogisticRegression(C=Cclass,random_state=1)
        model.fit(X,y)
        filterdcols = model.get_support()
        filterdcols = pd.Series(filterdcols).append(pd.Series(False),ignore_index=True)
        if type(predcolumn)==int:
            framenew = pd.concat([frame[frame.columns[filterdcols]],frame[frame.columns[predcolumn]]],axis=1)
        elif type(predcolumn)==str:
            framenew = pd.concat([frame[frame.columns[filterdcols]],frame[predcolumn]],axis=1)
        framenew = framenew.dropna()
    return framenew

## [Main Page](https://nikosga.github.io/)